# Оцінка моделей

In [1]:
import pandas as pd
import os
from sklearn.metrics import classification_report
from joblib import load

Зчитування файлу processed_data.csv.

In [3]:
df = pd.read_csv("processed_data.csv")

In [5]:
X = df[df.columns[1:-1]]
y = df[df.columns[-1:]].values.flatten()

__Функція для виведення класифікаційного звіту__

In [8]:
def model_classification_report(model, model_name: str, X_test, y_test):
    y_pred = model.predict(X_test)
    print(
        f"Класифікаційний звіт для моделі {model_name}:\n",
        classification_report(y_test, y_pred),
    )

__Функція для виведення розміру моделі__

In [11]:
def model_size(model_path: str):
    size_in_bytes = os.path.getsize(model_path)
    size_in_kb = size_in_bytes / 1024
    size_in_mb = size_in_kb / 1024
    print(
        f"Розмір моделі: {size_in_bytes} B ({size_in_kb:.2f} kB / {size_in_mb:.2f} MB)"
    )
    return size_in_bytes

__Функції для визначення важливості ознак моделі__

Для *Random Forest* і *Gradient Boosting*.

In [15]:
def model_weights(model) -> dict:
    """
    Функція повертає ваги усіх ознак.
    """
    weights_dict =  {
        name: weight
        for name, weight in zip(
            model.feature_names_in_, model.feature_importances_
        )
    }
    return dict(sorted(weights_dict.items(), key=lambda x: x[1], reverse=True))

In [17]:
def weight_importances(weights: dict):
    """
    Функція повертає важливі і неважливі ознаки в залежності від заданої точності.
    """
    is_very_important = {}
    is_important = {}
    not_very_important = {}
    not_important = {}
    for name, weight in zip(weights.keys(), weights.values()):
        if weight.round(0):
            is_very_important[name] = weight
        elif weight.round(1):
            is_important[name] = weight
        elif weight.round(2):
            not_very_important[name] = weight
        else:
            not_important[name] = weight
    return is_very_important, is_important, not_very_important, not_important

In [19]:
def weight_importances_report(model):
    """
    Функція виводить повний звіт з важливості ознак в залежності від заданої точності.
    """
    weights = model_weights(model)
    is_very_important, is_important, not_very_important, not_important = (
        weight_importances(weights)
    )
    print("Ваги всіх ознак:")
    for name, weight in zip(weights.keys(), weights.values()):
        print(f"{name}: {weight}")
    print("\n")

    print("Найважливіші ознаки:")
    for w in is_very_important.keys():
        print(w)
    print("\n")

    print("Значимі ознаки:")
    for w in is_important.keys():
        print(w)
    print("\n")

    print("Несуттєві ознаки:")
    for w in not_very_important.keys():
        print(w)
    print("\n")

    print("Неважливі ознаки:")
    for w in not_important.keys():
        print(w)

Завантаження моделей.

In [21]:
model_RF_path = "model_RandomForest.joblib"
model_HGB_path = "model_HistGradientBoosting.joblib"
model_GB_path = "model_GradientBoosting.joblib"
model_LR_path = "model_LogisticRegression.joblib"

In [23]:
model_RF = load(model_RF_path)
model_HGB = load(model_HGB_path)
model_GB = load(model_GB_path)
model_LR = load(model_LR_path)

In [25]:
model_RF_name = "Random Forest"
model_HGB_name = "Histogram-based Gradient Boosting Classification Tree"
model_GB_name = "Gradient Boosting"
model_LR_name = "Logistic Regression"

__Модель Random Forest__

In [27]:
model_classification_report(model_RF, model_RF_name, X, y)
model_RF_size = model_size(model_RF_path)

Класифікаційний звіт для моделі Random Forest:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99     32224
           1       1.00      0.98      0.99     40050

    accuracy                           0.99     72274
   macro avg       0.99      0.99      0.99     72274
weighted avg       0.99      0.99      0.99     72274

Розмір моделі: 35670553 B (34834.52 kB / 34.02 MB)


__Модель Histogram-based Gradient Boosting Classification Tree__

In [30]:
model_classification_report(model_HGB, model_HGB_name, X, y)
model_HGB_size = model_size(model_HGB_path)

Класифікаційний звіт для моделі Histogram-based Gradient Boosting Classification Tree:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98     32224
           1       1.00      0.97      0.98     40050

    accuracy                           0.98     72274
   macro avg       0.98      0.98      0.98     72274
weighted avg       0.98      0.98      0.98     72274

Розмір моделі: 1976064 B (1929.75 kB / 1.88 MB)


__Модель Gradient Boosting__

In [33]:
model_classification_report(model_GB, model_GB_name, X, y)
model_GB_size = model_size(model_GB_path)

Класифікаційний звіт для моделі Gradient Boosting:
               precision    recall  f1-score   support

           0       0.94      0.99      0.97     32224
           1       0.99      0.95      0.97     40050

    accuracy                           0.97     72274
   macro avg       0.97      0.97      0.97     72274
weighted avg       0.97      0.97      0.97     72274

Розмір моделі: 926285 B (904.58 kB / 0.88 MB)


__Модель Logistic Regression__

In [36]:
model_classification_report(model_LR, model_LR_name, X, y)
model_LR_size = model_size(model_LR_path)

Класифікаційний звіт для моделі Logistic Regression:
               precision    recall  f1-score   support

           0       0.90      0.84      0.87     32224
           1       0.88      0.93      0.90     40050

    accuracy                           0.89     72274
   macro avg       0.89      0.89      0.89     72274
weighted avg       0.89      0.89      0.89     72274

Розмір моделі: 1439 B (1.41 kB / 0.00 MB)


__Відсоткове співвідношення розмірів моделей__

In [39]:
sum_sizes = model_RF_size + model_HGB_size + model_GB_size + model_LR_size
print(f"{model_RF_name}: {model_RF_size / sum_sizes * 100:.2f}%")
print(f"{model_HGB_name}: {model_HGB_size / sum_sizes * 100:.2f}%")
print(f"{model_GB_name}: {model_GB_size / sum_sizes * 100:.2f}%")
print(f"{model_LR_name}: {model_LR_size / sum_sizes * 100:.2f}%")

Random Forest: 92.47%
Histogram-based Gradient Boosting Classification Tree: 5.12%
Gradient Boosting: 2.40%
Logistic Regression: 0.00%


__Визначення важливості ознак за допомогою моделей *Random Forest* і *Gradient Boosting*__

Модель *Random Forest*

In [43]:
weight_importances_report(model_RF)

Ваги всіх ознак:
remaining_contract: 0.7978162233940672
download_avg: 0.0887780831201279
subscription_age: 0.04147501909238496
bill_avg: 0.03381152413351241
upload_avg: 0.02509925171843683
is_movie_package_subscriber: 0.007645825767476941
service_failure_count: 0.003046797012352122
is_tv_subscriber: 0.0018317512409608222
download_over_limit: 0.0004955245206808143


Найважливіші ознаки:
remaining_contract


Значимі ознаки:
download_avg


Несуттєві ознаки:
subscription_age
bill_avg
upload_avg
is_movie_package_subscriber


Неважливі ознаки:
service_failure_count
is_tv_subscriber
download_over_limit


Модель *Gradient Boosting*

In [46]:
weight_importances_report(model_GB)

Ваги всіх ознак:
remaining_contract: 0.921066359947278
download_avg: 0.05571081374307791
bill_avg: 0.009974121926515287
subscription_age: 0.006614992250190256
upload_avg: 0.004121893959959762
is_movie_package_subscriber: 0.0013572162301059931
is_tv_subscriber: 0.0005196293915680024
service_failure_count: 0.00032580264163736665
download_over_limit: 0.00030916990966741017


Найважливіші ознаки:
remaining_contract


Значимі ознаки:
download_avg


Несуттєві ознаки:
bill_avg
subscription_age


Неважливі ознаки:
upload_avg
is_movie_package_subscriber
is_tv_subscriber
service_failure_count
download_over_limit


In [48]:
import json


# Шлях до файлу для збереження даних
results_path = "../results/model_evaluation.json"

# Функція для збереження даних у JSON-файл
def save_results_to_json(model_name, classification_report, model_size):
    # Перевіряємо, чи файл вже існує
    if os.path.exists(results_path):
        with open(results_path, "r") as f:
            results = json.load(f)
    else:
        results = {}

    # Додаємо дані для поточної моделі
    results[model_name] = {
        "classification_report": classification_report,
        "model_size_mb": model_size,
    }

    # Записуємо результати у JSON-файл
    with open(results_path, "w") as f:
        json.dump(results, f, indent=4)

# Приклад для Random Forest
model_RF_report = classification_report(y, model_RF.predict(X), output_dict=True)
model_RF_size_mb = model_size(model_RF_path) / (1024 * 1024)  # Конвертуємо в MB
save_results_to_json(model_RF_name, model_RF_report, model_RF_size_mb)

# Повторюємо для інших моделей
model_HGB_report = classification_report(y, model_HGB.predict(X), output_dict=True)
model_HGB_size_mb = model_size(model_HGB_path) / (1024 * 1024)
save_results_to_json(model_HGB_name, model_HGB_report, model_HGB_size_mb)

model_GB_report = classification_report(y, model_GB.predict(X), output_dict=True)
model_GB_size_mb = model_size(model_GB_path) / (1024 * 1024)
save_results_to_json(model_GB_name, model_GB_report, model_GB_size_mb)

model_LR_report = classification_report(y, model_LR.predict(X), output_dict=True)
model_LR_size_mb = model_size(model_LR_path) / (1024 * 1024)
save_results_to_json(model_LR_name, model_LR_report, model_LR_size_mb)


Розмір моделі: 35670553 B (34834.52 kB / 34.02 MB)


FileNotFoundError: [Errno 2] No such file or directory: '../results/model_evaluation.json'

## Висновки щодо вибору моделі

Розглянуто чотири моделі з бібліотеки `sklearn`. Здійснено оцінку *точності* моделей на усіх даних і на тестових даних за допомогою `classification_report` та *розміру* моделей.

**Рейтинг моделей за точність на тестових даних (починаючи від найточнішої):**

1. *Random Forest* та *Histogram-based Gradient Boosting Classification Tree*: 97%
2. *Gradient Boosting*: 96%
3. *Logistic Regression*: 89%

**Коментарі:** Моделі *Random Forest* та *Histogram-based Gradient Boosting Classification Tree* показали однакові значення метрик якості. Метод *Gradient Boosting* має трохи нижчі значення метрик якості. Модель *Logistic Regression* має відчутно меншу точність ніж усі інші, але однаково залишається досить точною.

**Рейтинг моделей за точність на усіх даних (починаючи від найточнішої):**

1. *Random Forest*: 99%
2. *Histogram-based Gradient Boosting Classification Tree*: 98%
3. *Gradient Boosting*: 97%
4. *Logistic Regression*: 89%

**Коментарі:** На усіх даних найточнішою є модель *Random Forest*. Точність моделі *Histogram-based Gradient Boosting Classification Tree* менше ніж *Random Forest*, але різниця невелика. У моделі *Gradient Boosting* нижче точність ніж у *Histogram-based Gradient Boosting Classification Tree*, але різниця також невелика. Модель *Logistic Regression* має відчутно меншу точність ніж усі інші, але однаково залишається досить точною.

**Рейтинг моделей за розміром (починаючи від найменшої):**

1. *Logistic Regression*: 1.41 kB (0.00% від суми розмірів усіх моделей)
2. *Gradient Boosting*: 0.88 MB (2.4% від суми розмірів усіх моделей)
3. *Histogram-based Gradient Boosting Classification Tree*: 1.88 MB (5.12% від суми розмірів усіх моделей)
4. *Random Forest*: 34.03 MB (92.47% від суми розмірів усіх моделей)

**Коментарі:** Модель *Random Forest* важить набагато більше ніж інші моделі. Модель *Gradient Boosting* важить менше ніж *Histogram-based Gradient Boosting Classification Tree*, але різниця не є суттєвою. Модель *Logistic Regression* займає мінімальну кількість пам'яті.

**Підсумуємо результати:**

- Найточнішою є модель *Random Forest*, але й важить вона набагато більше ніж інші моделі.
- Точність моделі *Histogram-based Gradient Boosting Classification Tree* трохи менше ніж у моделі *Random Forest*, але важить вона набагато менше. Також саме ця модель найкраще себе показала на тестових даних.
- Модель *Gradient Boosting* має трохи нижчу точність та розмір ніж модель *Histogram-based Gradient Boosting Classification Tree*, але різниця не є суттєвою.
- Модель *Logistic Regression* має відчутно меншу точність ніж усі інші, але однаково залишається досить точною. До того ж модель займає мінімальну кількість пам'яті. Для *задачі прогнозування відтоку клієнтів* висока точність не є критично необхідною, тому модель має право на існування.

**Рекомендації:**

- Якщо обирати "золоту середину" між точністю та розміром, то варто обрати *Histogram-based Gradient Boosting Classification Tree*. Модель має високу точність і невеликий розмір.
- Якщо критично важлива точність, то варто обрати *Random Forest*.
- Якщо критично важливим є розмір, то варто обрати *Logistic Regression*.

## Висновки щодо важливості ознак

**Рейтинг ознак:**

- *Найважливіша:*
1. `reamining_contract`

- *Значима:*

2. `download_avg`

- *Несуттєві:*

3. `subscription_age`

4. `bill_avg`

- *Майже несуттєві:*

5. `upload_avg`

6. `is_movie_package_subscriber`

- *Неважливі:*

7. `service_failure_count`

8. `is_tv_subscriber`

9. `download_over_limit`

Отже, відток клієнтів **сильно залежить** від *терміну залишку контракту* і **незалежить** від *кількісті збоїв сервісу*, *наявності підписки на ТБ* та *перевищення ліміту завантаження*. Також відток клієнтів **помірно залежить** від *середньої швидкості завантаження*, **слабо залежить** від *тривалості підписки* і *розміру середнього рахунку* та **майже незалежить** від *середньої швидкості вивантаження* і *наявності пакету фільмів*.